A. Read the  dataset from the following random_sampled_data.csv Links to an external site. and assign it to a Pandas DataFrame as you learned in tutorial Lab2-3.

In [36]:
# The following line will import KNeighborsClassifier "Class"
# KNeighborsClassifier is name of a "sklearn class" to perform "KNN Classification" 

from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random
# reading a CSV file directly from Web, and store it in a pandas DataFrame:
# "read_csv" is a pandas function to read csv files from web or local device:

# Set the total number of rows in the CSV file
#total_rows = 852950  # You should replace this with the actual total number of rows in your dataset

# Number of rows you want to read
#num_rows = 1000

# Include the first row and randomly sample the remaining rows
#random_rows = [0] + random.sample(range(1, total_rows), num_rows - 1)

# Read the CSV file for the specified random rows
#df = pd.read_csv('../Final project/Crime_Data_from_2020_to_Present.csv', skiprows=lambda x: x not in random_rows)

#df.to_csv("random_sampled_data.csv", index=False)

df = pd.read_csv("random_sampled_data.csv")

df_cleaned = df.dropna()


# Print the DataFrame
print(df)

         DR_NO               Date Rptd                DATE OCC  TIME OCC  \
0    200104875  01/12/2020 12:00:00 AM  01/12/2020 12:00:00 AM      1955   
1    200816285  11/11/2020 12:00:00 AM  11/10/2020 12:00:00 AM      1700   
2    200304982  01/17/2020 12:00:00 AM  01/17/2020 12:00:00 AM      1825   
3    200504796  01/19/2020 12:00:00 AM  01/19/2020 12:00:00 AM      1800   
4    200316294  08/25/2020 12:00:00 AM  08/25/2020 12:00:00 AM       700   
..         ...                     ...                     ...       ...   
994  230616403  10/12/2023 12:00:00 AM  10/11/2023 12:00:00 AM      1420   
995  231407108  03/16/2023 12:00:00 AM  03/15/2023 12:00:00 AM      2200   
996  231815151  08/18/2023 12:00:00 AM  08/17/2023 12:00:00 AM      2300   
997  230509664  06/07/2023 12:00:00 AM  06/07/2023 12:00:00 AM      1400   
998  231404405  01/12/2023 12:00:00 AM  01/12/2023 12:00:00 AM      2235   

     AREA  AREA NAME  Rpt Dist No  Part 1-2  Crm Cd  \
0       1    Central          17

##### B. Now, we want to use the categorical features as well! To this end, we have to perform a feature engineering process called OneHotEncoding for the categorical features. To do this, each categorical feature should be replaced with dummy columns in the feature table (one column for each possible value of a categorical feature), and then encode it in a binary manner such that only one of the dummy columns can take “1” at a time (and zero for the rest).

In [46]:

# Creating the Feature Matrix for dataset:
new_x = df


# Function to map month to season
def get_season(month):
    if 3 <= month <= 5:
        return 'Spring'
    elif 6 <= month <= 8:
        return 'Summer'
    elif 9 <= month <= 11:
        return 'Fall'
    else:
        return 'Winter'


# Extracting the month from the 'Date' column and mapping it to seasons
#new_x['Month'] = pd.to_datetime(new_x['date_occ']).dt.month

new_x['Month'] = pd.to_datetime(new_x['DATE OCC']).dt.month
new_x['Season'] = new_x['Month'].apply(get_season)



# Clean df of nah, Vict Sex =X , and age = 0 
new_x.dropna(inplace=True, subset= ['Vict Age','Vict Sex','Vict Descent'])
new_x = new_x[new_x['Vict Sex'] != 'X'].reset_index(drop=True)
new_x = new_x[new_x['Vict Descent'] != 'X'].reset_index(drop=True)
new_x = new_x[new_x['Vict Age'] !=  0].reset_index(drop=True)

#Add categorical features as 1 or 0 for seasons
new_x['Spring'] = np.where((new_x['Season'] == 'Spring'), 1,0)
new_x['Summer'] = np.where((new_x['Season'] == 'Summer'), 1,0)
new_x['Fall'] = np.where((new_x['Season'] == 'Fall'), 1,0)
new_x['Winter'] = np.where((new_x['Season'] == 'Winter'), 1,0)

#Add categorical featrues as 1 or 0 for Vict Sex
new_x['Male'] = np.where((new_x['Vict Sex'] == 'M'), 1,0)
new_x['Female'] = np.where((new_x['Vict Sex'] == 'F'), 1,0)

#Add categorical features as 1 or 0 for Vict Descent
# new_x['Hispanic'] = np.where((new_x['Vict Descent'] == 'H'), 1,0)
# new_x['Black'] = np.where((new_x['Vict Descent'] == 'B'), 1,0)
# new_x['White'] = np.where((new_x['Vict Descent'] == 'W'), 1,0)
# new_x['Hispanic'] = np.where((new_x['Vict Descent'] == 'H'), 1,0)

# create a python list of feature names that would like to pick from the dataset:
feature_cols = [
    'Vict Age',
    'Spring',
    'Summer',
    'Fall',
    'Winter',
    'Male',
    'Female',
    ]

# use the above list to select the features from the original DataFrame
X = new_x[feature_cols]
# select a Series of labels (the last column) from the DataFrame
y = new_x['Crm Cd Desc'] # this is the original categorical labels (the latest revision of sklearn accepts non-numerical labels)



with pd.option_context('display.max_rows', None,'display.max_columns', None,'display.precision', 3,):
    print(X)

     Vict Age  Spring  Summer  Fall  Winter  Male  Female
0          31       0       0     1       0     1       0
1          38       0       0     0       1     1       0
2          27       0       1     0       0     1       0
3          48       0       0     1       0     1       0
4          52       0       0     0       1     1       0
5          35       0       0     0       1     1       0
6          21       0       1     0       0     1       0
7          12       0       0     1       0     0       1
8          63       0       0     1       0     1       0
9          26       0       0     1       0     0       1
10         41       0       1     0       0     0       1
11         48       0       0     0       1     1       0
12         21       0       0     1       0     1       0
13         58       0       0     1       0     0       1
14         54       0       1     0       0     0       1
15         89       0       0     0       1     0       1
16         17 

C.Split the dataset into testing and training sets with the following parameters: test_size=0.4, random_state=10

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=10)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(438, 7)
(293, 7)
(438,)
(293,)


##### D. Now, normalize (scale) the features. To normalize the data, you can simply use StandardScaler  from sklearn. (note: don’t normalize the target!). Remember that we can only use the statistics of X_train for normalization, and then apply it on both X_train and X_test:

In [48]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

E.Instantiate a KNN object with K=3, train it on the training set and test it on the testing set. Then, calculate the accuracy of your prediction as you learned in Lab3.

In [51]:

k = 3

my_knn_for_cs4661 = KNeighborsClassifier(n_neighbors=k, weights='uniform') # name of the object is arbitrary!
my_knn_for_cs4661.fit(X.values, y)


### Training ONLY on the training set:
# Training ONLY on the training set:

my_knn_for_cs4661.fit(X_train_normalized, y_train)

### Testing on the testing set:
# Testing on the testing set:

y_predict = my_knn_for_cs4661.predict(X_test_normalized)


accuracy = accuracy_score(y_test, y_predict)

print(accuracy)


0.09215017064846416


D.Repeat part (c) for K=1, K=3, K=5, K=11, K=15, K=27, K=59 (you can simply use a “for loop,” and save the final accuracy results in a list). Does the accuracy always get better by increasing the value K?

In [52]:

K_values = [1,3,5,11,15,27,59]
result_list = []

for K in K_values:

    my_knn_for_cs4661 = KNeighborsClassifier(n_neighbors=K, weights='uniform') # name of the object is arbitrary!
    my_knn_for_cs4661.fit(X.values, y)


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=10) # We can fix the random_state for reproducibility!

### Training ONLY on the training set:
# Training ONLY on the training set:

    my_knn_for_cs4661.fit(X_train, y_train)

### Testing on the testing set:
# Testing on the testing set:

    y_predict = my_knn_for_cs4661.predict(X_test)


    accuracy = accuracy_score(y_test, y_predict)
    result_list.append(accuracy)
print(result_list)

[0.10580204778156997, 0.08532423208191127, 0.10238907849829351, 0.08873720136518772, 0.09897610921501707, 0.08873720136518772, 0.07849829351535836]


E.Now, suppose that we would like to make prediction based on only ONE single feature. To find the best single feature, we have to try every feature individually. In other word, we have to repeat part (c) with K=11, four times (each time using only one of the 4 features), and compute the accuracy each time. Then, check the accuracies. 
Which individual feature provide the best accuracy (the best feature)?  Which one is the second best feature?

In [55]:
feature_cols = [
    'Spring',
    'Summer',
    'Fall',
    'Winter',
    ]

# use the above list to select the features from the original DataFrame
X = new_x[feature_cols]

# select a Series of labels (the last column) from the DataFrame
# y = iris_df['label'] # this is the index that we gave to the labels
# OR:
y = new_x['Crm Cd Desc']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=10)

k = 11

my_knn_for_cs4661 = KNeighborsClassifier(n_neighbors=k, weights='uniform') # name of the object is arbitrary!
my_knn_for_cs4661.fit(X.values, y)


### Training ONLY on the training set:
# Training ONLY on the training set:

my_knn_for_cs4661.fit(X_train, y_train)

### Testing on the testing set:
# Testing on the testing set:

y_predict = my_knn_for_cs4661.predict(X_test)


accuracy = accuracy_score(y_test, y_predict)

print(accuracy)

0.09215017064846416


In [61]:
feature_cols = [
    'Vict Age',
    ]

# use the above list to select the features from the original DataFrame
X = new_x[feature_cols]

# select a Series of labels (the last column) from the DataFrame
# y = iris_df['label'] # this is the index that we gave to the labels
# OR:
y = new_x['Crm Cd Desc']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=10)

k = 11

my_knn_for_cs4661 = KNeighborsClassifier(n_neighbors=k, weights='uniform') # name of the object is arbitrary!
my_knn_for_cs4661.fit(X.values, y)


### Training ONLY on the training set:
# Training ONLY on the training set:

my_knn_for_cs4661.fit(X_train, y_train)

### Testing on the testing set:
# Testing on the testing set:

y_predict = my_knn_for_cs4661.predict(X_test)


accuracy = accuracy_score(y_test, y_predict)

print(accuracy)

###############################################################################

feature_cols = [
    'Spring',
    'Summer',
    'Fall',
    'Winter',
    ]

# use the above list to select the features from the original DataFrame
X = new_x[feature_cols]


# select a Series of labels (the last column) from the DataFrame
# y = iris_df['label'] # this is the index that we gave to the labels
# OR:
y = new_x['Crm Cd Desc']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=10)


my_knn_for_cs4661 = KNeighborsClassifier(n_neighbors=k, weights='uniform') # name of the object is arbitrary!
my_knn_for_cs4661.fit(X.values, y)


### Training ONLY on the training set:
# Training ONLY on the training set:

my_knn_for_cs4661.fit(X_train, y_train)

### Testing on the testing set:
# Testing on the testing set:

y_predict = my_knn_for_cs4661.predict(X_test)


accuracy = accuracy_score(y_test, y_predict)

print(accuracy)

#################################################################################
feature_cols = [
    'Male',
    'Female',
    ]

# use the above list to select the features from the original DataFrame
X = new_x[feature_cols]

# select a Series of labels (the last column) from the DataFrame
# y = iris_df['label'] # this is the index that we gave to the labels
# OR:
y = new_x['Crm Cd Desc']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=10)


my_knn_for_cs4661 = KNeighborsClassifier(n_neighbors=k, weights='uniform') # name of the object is arbitrary!
my_knn_for_cs4661.fit(X.values, y)


### Training ONLY on the training set:
# Training ONLY on the training set:

my_knn_for_cs4661.fit(X_train, y_train)

### Testing on the testing set:
# Testing on the testing set:

y_predict = my_knn_for_cs4661.predict(X_test)


accuracy = accuracy_score(y_test, y_predict)

print(accuracy)



0.07167235494880546
0.09215017064846416
0.08191126279863481


Which individual feature provide the best accuracy (the best feature)? 

petal_width and petal_length are tied for the best features 

E.Now, we want to repeat part (e) (with K=11), this time using two features. you have to train, test, and evaluate your model for 6 different cases: using (1st and 2nd features), (1st and 3rd features), (1st and 4th  features), (2nd  and 3rd  features), (2nd and 4th features), (3rd and 4th  features)!    
Which “feature pair” provides the best accuracy?

In [58]:

def generate_combinations(items):
    combinations_list = []
    n = len(items)
    for i in range(n):
        for j in range(i + 1, n):
            combinations_list.append([items[i], items[j]])
    return combinations_list

feature_cols = [
    'Vict Age',
    'Spring',
    'Summer',
    'Fall',
    'Winter',
    'Male',
    'Female',
    ]

# Generate and print unique combination lists
unique_combination_lists = generate_combinations(feature_cols)
print("Unique combination lists:")
for combo_list in unique_combination_lists:
    print(combo_list)

    # use the above list to select the features from the original DataFrame
    X = new_x[combo_list]

    # select a Series of labels (the last column) from the DataFrame
    # y = iris_df['label'] # this is the index that we gave to the labels
    # OR:
    y = new_x['Crm Cd Desc']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=10)


    my_knn_for_cs4661 = KNeighborsClassifier(n_neighbors=k, weights='uniform') # name of the object is arbitrary!
    my_knn_for_cs4661.fit(X.values, y)


    ### Training ONLY on the training set:
    # Training ONLY on the training set:

    my_knn_for_cs4661.fit(X_train, y_train)

    ### Testing on the testing set:
    # Testing on the testing set:

    y_predict = my_knn_for_cs4661.predict(X_test)

    accuracy = accuracy_score(y_test, y_predict)

    print(accuracy)

Unique combination lists:
['Vict Age', 'Spring']
0.09215017064846416
['Vict Age', 'Summer']


0.08191126279863481
['Vict Age', 'Fall']
0.05802047781569966
['Vict Age', 'Winter']
0.08873720136518772
['Vict Age', 'Male']
0.06825938566552901
['Vict Age', 'Female']
0.06825938566552901
['Spring', 'Summer']
0.09556313993174062
['Spring', 'Fall']
0.07849829351535836
['Spring', 'Winter']
0.08191126279863481
['Spring', 'Male']
0.08191126279863481
['Spring', 'Female']
0.08191126279863481
['Summer', 'Fall']
0.06143344709897611
['Summer', 'Winter']
0.05460750853242321
['Summer', 'Male']
0.07508532423208192
['Summer', 'Female']
0.06484641638225255
['Fall', 'Winter']
0.08191126279863481
['Fall', 'Male']
0.06143344709897611
['Fall', 'Female']
0.10238907849829351
['Winter', 'Male']
0.10580204778156997
['Winter', 'Female']
0.10580204778156997
['Male', 'Female']
0.08191126279863481
